## Capstone - project ( dataframe ) 
This project consists of scraping a wikipedia page and generating a dataframe. First we will read all the data from the page and systematically put them into the respective columns of Postal code, Borough and Neighborhood ( of the dataframe to be generated ). In the first step al the respective libraries are downloaded

In [70]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

The above cell takes care of the necessary libraries that need to be imported. The next step is to download the link and store the URL into the object wikipedia_link

In [71]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [72]:
raw_wikipedia_page= requests.get(wikipedia_link)
url  = requests.get(wikipedia_link).text
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())


The raw data is stored as a text file. The table class find is done in next steps and the postal codes are extracted. Avoiding index errors, all the rows are first extracted irrespective of their values ( whether assigned or not ). Neighbourhood columns are split using the delimiter '(' or the 2nd index of tag 'a' in the html script

In [73]:
# code to find the respective columns
my_table = soup.find('table')
list_of_rows = []
Neighbourhood = []
Borough       = []
postcode      = []
notpostcode   = [] # the column list to subtract from the original columnof unassigned values
#cell by cell check for tag 'td' and 'title', extracting the span text value. 
for cell in my_table.find_all('td'):
    postcode.append(cell.find_all('b')[0].text)
    if cell.find_all('title') is None: # for the cases where there are no titles
        a1=cell.find_all('span')[0].text
        Borough.append(a1.split('(',1)[0])# cases where the span text value has bracketed value ( which is the neighboru value)
        
    else:
        try:
            a1=cell.find_all('span')[0].text # total span value
            a2=cell.find_all('a')[1].text # neighbourhood value 
            a3=a1.replace(a2,"")
            a4=a3.split('(',1)[0]
            Borough.append(a4)
           
        except IndexError:          # for index error, in cases where the neighbourhood values do not exist as the value of cell is taken as borough value
            a1=cell.find_all('span')[0].text
            a2=a1.split('(',1)[0]
            Borough.append(a2)
    for notas in cell.find_all('i'):
        notpostcode.append(cell.find_all('b')[0].text)

#cell extraction for the neighbourhood 
for cell in my_table.find_all('td'):
    if not not cell.find_all('i'):
        Neighbourhood.append(cell.find_all('span')[0].text)
    elif cell.find_all('a') is None:
        try:
            a1=cell.find_all('span')[0].text
            Neighbourhood.append(a1.split('(',1)[1])
        except IndexError:
            a1='Not assigned'
            Neighbourhood.append(a1) # for Indexerror if there isn't any split value
    else:
        try:
            a1=cell.find_all('span')[0].text # neighbourhood value
            Neighbourhood.append(a1.split('(',1)[1])
        except IndexError:
            a1='Not assigned'
            Neighbourhood.append(a1)


The following part of the algorithm manually cleans the leftover data

In [74]:
# Cleaning the columns and manually replacing values from Borough to Neighbourhood columns ( 2-3 nos) which had no space (  ) between Borough and Neighbourhood value; for eg
# MississaugaCanada Post Gateway Processing Centre
b_indices = [i for i, s in enumerate(Borough) if '\n' in s]
Borough = [borough.replace('\n','') for borough in Borough]

b1_index= [i for i, s in enumerate(Borough) if 'Mississauga' in s]
Neighbourhood[b1_index[0]]=Borough[b1_index[0]][11:] 
Borough[b1_index[0]]=Borough[b1_index[0]][:11]

b2_index= [i for i, s in enumerate(Borough) if 'PO Boxes25' in s]
Neighbourhood[b2_index[0]]=Borough[b2_index[0]][16:] 
Borough[b2_index[0]]=Borough[b2_index[0]][:16]

b3_index= [i for i, s in enumerate(Borough) if 'EtobicokeNorthwest' in s]
Neighbourhood[b3_index[0]]=Borough[b3_index[0]][9:] 
Borough[b3_index[0]]=Borough[b3_index[0]][:9]

b4_index= [i for i, s in enumerate(Borough) if 'East YorkEast Toronto' in s]
Neighbourhood[b4_index[0]]=Borough[b4_index[0]][9:] 
Borough[b4_index[0]]=Borough[b4_index[0]][:9]

b5_index= [i for i, s in enumerate(Borough) if 'TorontoBusiness' in s]
Neighbourhood[b5_index[0]]=Borough[b5_index[0]][12:] 
Borough[b5_index[0]]=Borough[b5_index[0]][:12]


Clean the Neighbourhood columns 

In [75]:
Neighbourhood = [n.replace(')','') for n in Neighbourhood]
Neighbourhood = [n.replace('/',',') for n in Neighbourhood]
Neighbourhood = [n.replace('(',',') for n in Neighbourhood]

Generate DataFrame and delete the columns with Not assigned value as in Borough

In [76]:
#passing the data to dataframe
df = pd.DataFrame()
df['Postcode']=postcode
df['Borough']= Borough
df['Neighbourhood']=Neighbourhood
df_post = df[ df.Borough != 'Not assigned']
df_post.reset_index(drop=True)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don MillsNorth
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [78]:
df_post.shape # final dataframe shape

(103, 3)